In [ ]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

!pip install Faker
from faker import Faker
import sys
import os
import random
from tqdm import tqdm
import tensorflow as tf
import numpy as np
from babel.dates import format_date

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
os.listdir('/tmp')

['lines', 'wordembeddings']

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
os.mkdir("/tmp/wordembeddings")

--2020-06-24 17:27:23--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-06-24 17:27:23--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-06-24 17:27:23--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip glove.6B.zip -d /tmp/wordembeddings
os.listdir('/tmp/wordembeddings')

Archive:  glove.6B.zip
replace /tmp/wordembeddings/glove.6B.50d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

['glove.6B.300d.txt',
 'glove.6B.200d.txt',
 'glove.6B.50d.txt',
 'glove.6B.100d.txt']

In [ ]:
wordembfile = open('/tmp/wordembeddings/glove.6B.100d.txt', "r")
# wordembd = wordembfile.read()
print((wordembfile.readline()))

the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062



In [ ]:
"""This is a new collection of translated movie subtitles from http://www.opensubtitles.org/.
IMPORTANT: If you use the OpenSubtitle corpus:
Please, add a link to http://www.opensubtitles.org/ to your website and to your reports and publications produced with the data! I promised this when I got the data from the providers of that website!"""
!wget http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz

--2020-06-24 20:00:21--  http://opus.nlpl.eu/download.php?f=OpenSubtitles/v2018/mono/OpenSubtitles.raw.en.gz
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz [following]
--2020-06-24 20:00:21--  https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/en.txt.gz
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3663376519 (3.4G) [application/gzip]
Saving to: ‘download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz.1’

download.php?f=Open 100%[===================>]   3.41G  22.4MB/s    in 2m 39s  

2020-06-24 20:03:01 (22.0 MB/s) - ‘download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.

In [ ]:
!gunzip -k download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en.gz
# os.mkdir('/tmp/lines')
!split -a 3 -l 100000 download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en /tmp/lines/lines-

gzip: download.php?f=OpenSubtitles%2Fv2018%2Fmono%2FOpenSubtitles.raw.en already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [ ]:
os.listdir('/tmp/lines')
# print(len(os.listdir('/tmp/lines')))

['lines-fem',
 'lines-dbj',
 'lines-dws',
 'lines-bqd',
 'lines-cep',
 'lines-gdy',
 'lines-fee',
 'lines-gbh',
 'lines-buo',
 'lines-etf',
 'lines-die',
 'lines-cmw',
 'lines-eqi',
 'lines-exc',
 'lines-dwb',
 'lines-dzw',
 'lines-aum',
 'lines-emf',
 'lines-gaf',
 'lines-djj',
 'lines-dxm',
 'lines-bny',
 'lines-gkw',
 'lines-gmj',
 'lines-bvz',
 'lines-aco',
 'lines-fbp',
 'lines-ase',
 'lines-gcw',
 'lines-evt',
 'lines-aen',
 'lines-buv',
 'lines-clw',
 'lines-gck',
 'lines-ctj',
 'lines-dnj',
 'lines-fwv',
 'lines-cot',
 'lines-duy',
 'lines-cqy',
 'lines-eql',
 'lines-acf',
 'lines-fuy',
 'lines-eot',
 'lines-eqs',
 'lines-gcy',
 'lines-bop',
 'lines-bxn',
 'lines-dqy',
 'lines-bdy',
 'lines-fua',
 'lines-bhw',
 'lines-awx',
 'lines-dpu',
 'lines-fmp',
 'lines-csg',
 'lines-fka',
 'lines-azk',
 'lines-bkw',
 'lines-cyr',
 'lines-dkf',
 'lines-bff',
 'lines-elo',
 'lines-ent',
 'lines-bex',
 'lines-dwv',
 'lines-djl',
 'lines-fdg',
 'lines-ggq',
 'lines-aaa',
 'lines-cuf',
 'line

In [ ]:
texts = []
# labels_index = {}  # dictionary mapping label name to numeric id
# labels = []  # list of label ids
for file in (os.listdir('/tmp/lines'))[:50]:
  path = os.path.join('/tmp/lines', file)
  # label_id = len(labels_index)
  # labels_index[file] = label_id
  if sys.version_info < (3,):
    f = open(path)
  else:
    f = open(path, encoding='latin-1')
  for line in f:
    stripped_line = line.strip()
    texts.append(stripped_line)
    # print(stripped_line)
  # t = f.read()
  # i = t.find('\n')  # skip header
  # if 0 < i:
  #   # t = t[i:]
  #   with open("/tmp/lines/lines-alu", "r") as file:
  # for line in file:
  #   stripped_line = line.strip()
  #   print(stripped_line)
  f.close()
  # print(c)
  # labels.append(label_id)
# print(labels)

In [ ]:
print(len(texts))
# print(texts[0])

5000000


check for labels sparse_categorical_crossentropy nd splitting into train nd val

In [ ]:
msg = []
reply = []
for file in (os.listdir('/tmp/lines'))[:50]:
  path = os.path.join('/tmp/lines', file)
  tfile = open(path,'r')
  txt = tfile.readlines()
  for i,lines in enumerate(txt[:-1]):
    lines = lines.strip('\n')
    lines = lines.lower()
    lines = lines.replace(".", "")
    lines = lines.replace(",", "")
    lines = lines.replace("?", "")
    lines = lines.replace("!", "")
    lines = lines.replace(":", "")
    lines = lines.replace(">", "")
    lines = lines.replace("<", "")
    lines = lines.replace("-", " ")
    lines = lines.replace("_", " ")
    lines = lines.replace("\\", "")
    lines = lines.replace("  ", " ")
    lines = "startseq " + lines + " endseq"
    lines2 = "startseq " + txt[i+1] + " endseq"
    # print(lines)
    msg.append(lines)
    reply.append(txt[i+1])
  tfile.close()

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words= 5000, oov_token = "<OOV>", lower=True, split=' ')
tokenizer.fit_on_texts(msg)

# # labels = to_categorical(np.asarray(labels))
# print('Shape of data tensor:', data.shape)
# # print('Shape of label tensor:', labels.shape)

# # split the data into a training set and a validation set
# indices = np.arange(data.shape[0])
# np.random.shuffle(indices)
# data = data[indices]
# # labels = labels[indices]
# print(data.shape)
# nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

# x_train = data[:-nb_validation_samples]
# y_train = labels[:-nb_validation_samples]
# x_val = data[-nb_validation_samples:]
# y_val = labels[-nb_validation_samples:]

In [ ]:
# sequences = tokenizer.texts_to_sequences(texts[:50000])

word_index = tokenizer.word_index
# print(type(word_index))
print('Found %s unique tokens.' % len(word_index))
# data = pad_sequences(sequences, padding='post')

Found 130590 unique tokens.


later split data into train nd valid

In [ ]:
embeddings_index = {}
f = open(os.path.join('/tmp/wordembeddings', 'glove.6B.100d.txt'))
for Line in f:
    values = Line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))
print(embeddings_index['end'])

Found 400000 word vectors.
[ 0.14726   0.088411  0.28386  -0.2797   -0.32498  -0.31503  -0.16317
  0.49901  -0.050685 -0.13406   0.18811   0.17244   0.19983   0.106
  0.44888  -0.16482  -0.15723  -0.2267   -0.52985  -0.19136   0.66418
 -0.34908   0.2041    0.19812   0.96231  -0.076071 -0.40823  -0.54391
  0.4698   -0.42401  -0.20947   0.25178   0.081003 -0.54254  -0.15188
  0.16457   0.018161 -0.074509 -0.19323  -0.27414  -0.060197 -0.16855
  0.37944   0.015304 -0.10552   0.025971  0.23706  -0.46123   0.22297
 -0.83349   0.15802  -0.20303  -0.12614   1.0924   -0.28469  -2.8841
 -0.48502  -0.70376   1.9497    0.3812   -0.42812   0.48972  -0.41521
 -0.43973   0.66411  -0.094294  0.52652   0.090762 -0.28974  -0.43982
  0.088801 -0.33429  -0.77197  -0.60414  -0.077605 -0.34217  -0.68175
  0.72201  -0.93264  -0.23963   0.43971   0.08899  -0.44907   0.48743
 -1.0736    0.32468  -0.11632  -0.2534    0.23838  -0.18391  -0.27039
  0.72632  -0.61252   0.56877  -0.7907   -0.37271   0.28074   0.28

In [ ]:
embedding_matrix = np.zeros((5001, 100))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  # print(embedding_vector)
  if embedding_vector is not None:
    # words not found in embedding index will be all-zeros.
    embedding_matrix[i] = embedding_vector
  if i == 5000:
    break

from keras.layers import Embedding
embedding_layer = Embedding(5001, 100, weights=[embedding_matrix], input_length=20, trainable=False)
#max_sequence_length = 20 was found by printing shape of data in code above

In [ ]:
sequences = tokenizer.texts_to_sequences(msg[:10000])
inputmsg = pad_sequences(sequences, padding='post', maxlen =20, truncating='post')
sequences = tokenizer.texts_to_sequences(reply[:10000])
outputlabels = pad_sequences(sequences, padding='post', maxlen =20, truncating='post')

In [ ]:
print(inputmsg.shape)
print(outputlabels.shape)
print(len(msg))
print(inputmsg[0])
print(type(inputmsg[0][0]))
print(outputlabels[0])
print(msg[0])
print(reply[0])

(10000, 20)
(10000, 20)
4999950
[  2  16  72   4 100   3   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]
<class 'numpy.int32'>
[   5 2311  203  256    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]
startseq what did you say endseq
The governor doesn't care.



In [ ]:
for word, val in word_index.items():  
  if word == "end":
    print(val)
  if word == "endseq":
    print(val)
  if word == "startseq":
    print(val)
  if val == 88:
    print(word)

2
3
take
332


the code shows some puctuations 
would it affect?
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
later try this


In [ ]:
print(embeddings_index['the'])

[-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0.28787  -0.06731   0.30906  -0.26384
 -0.13231  -0.20757   0.33395  -0.33848  -0.31743  -0.48336   0.1464
 -0.37304   0.34577   0.052041  0.44946  -0.46971   0.02628  -0.54155
 -0.15518  -0.14107  -0.039722  0.28277   0.14393   0.23464  -0.31021
  0.086173  0.20397   0.52624   0.17164  -0.082378 -0.71787  -0.41531
  0.20335  -0.12763   0.41367   0.55187   0.57908  -0.33477  -0.36559
 -0.54857  -0.062892  0.26584   0.30205   0.99775  -0.80481  -3.0243
  0.01254  -0.36942   2.2167    0.72201  -0.24978   0.92136   0.034514
  0.46745   1.1079   -0.19358  -0.074575  0.23353  -0.052062 -0.22044
  0.057162 -0.15806  -0.30798  -0.41625   0.37972   0.15006  -0.53212
 -0.2055   -1.2526    0.071624  0.70565   0.49744  -0.42063   0.26148
 -1.538    -0.30223  -0.073438 -0.28312   0.37104  -0.25217   0.016215
 -0.017099 -0.38984   0.87424  -0.72569  -0.51058  -0.52028  -0.1459
  0.8278    0.27062 ]

In [ ]:
inputmsg = tf.convert_to_tensor(inputmsg, dtype = np.int32)
emb_inputseq = embedding_layer(inputmsg)
print(emb_inputseq.shape)
print(K.is_keras_tensor(emb_inputseq))
# inputmsg = Lambda(lambda x: x[:,:])(inputmsg)
print(K.is_keras_tensor(inputmsg))

(10000, 20, 100)
True
False


In [ ]:
n_s=256
post_activation_LSTM_cell = LSTM(n_s, return_state = True) # post-attention LSTM 
outputs = []
m = 10000
max_seq_length = 20
wordemb=100
repeater = RepeatVector(max_seq_length)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "relu")
densor2 = Dense(1, activation = "relu")
activator = Activation('tanh', name='attention_weights')
dotor = Dot(axes = 1)
output_layer = Dense(5000, activation='softmax')

def one_att(h, s_prev):
  s_prev = repeater(s_prev)
  concat = concatenator(inputs=[h,s_prev])
  intmdten = densor1(concat)
  energies = densor2(intmdten)
  alphas = activator(energies)
  context_t = dotor([alphas, h])
  return context_t
    #s0=encoded sentence
  # def pre_att():
    
  #   return h
  #   print(h[])
  #   # context = Dense(1)
def post_att():
  X=Input(shape =(20,))
  print(X.shape)
  emb_inputseq = embedding_layer(X)
  h = Bidirectional(LSTM(units = 128, return_sequences=True))(emb_inputseq)
  # print(K.is_keras_tensor(h))
  # print(h[:, 19, :])
  s = Lambda(lambda x: x[:,19,:])(h)
  # print(type(s))
  # print(s.shape)
  # print(K.is_keras_tensor(s))
  c0 = Input(shape=(n_s,), name='c')
  c = c0
  print(c.shape)
  # print(K.is_keras_tensor(c))
  # print(type(c))
  for t in range(20):
    context_t = one_att(h, s)
    # print(K.is_keras_tensor(context_t))
    # print(type(context_t))
    s, _, c = post_activation_LSTM_cell(inputs=context_t, initial_state=[s, c])
    # output_layer = Dense(100, activation=softmax)
    out = output_layer(s)
    outputs.append(out)
  model = Model(inputs=[X,c0], outputs=outputs)  
  return model
  

try using pad_seq() for converting replies into s nd finding next word

In [ ]:
model = post_att()
model.summary()

(None, 20)
(None, 256)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         multiple             500100      input_5[0][0]                    
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 20, 256)      234496      embedding_8[1][0]                
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, 256)          0           bidirectional_5[0][0]            
_____________________________________________________________________

In [ ]:
opt = Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999,  decay = 0.001)
model.compile(optimizer = opt, loss = "sparse_categorical_crossentropy", metrics = ['accuracy'])

In [ ]:
c1 = np.zeros((10000,256))
print(type(c1))
print(type(inputmsg))
inputmsg=np.array(inputmsg)
print(type(outputlabels))

outputlabels = list(outputlabels.swapaxes(0,1))

#session crash due to this
# outputlabels = to_categorical(np.asarray(outputlabels))


model.fit([inputmsg,c1], outputlabels, batch_size = 100, epochs = 5)
# resp = model.predict()

<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'numpy.ndarray'>
Epoch 1/5
10000/10000 [==============================] - 112s 11ms/step - loss: 64.5396 - dense_21_loss: 1.6271 - dense_21_accuracy: 0.0131 - dense_21_accuracy_1: 0.1434 - dense_21_accuracy_2: 0.2548 - dense_21_accuracy_3: 0.3559 - dense_21_accuracy_4: 0.4570 - dense_21_accuracy_5: 0.5550 - dense_21_accuracy_6: 0.6327 - dense_21_accuracy_7: 0.7017 - dense_21_accuracy_8: 0.7565 - dense_21_accuracy_9: 0.8007 - dense_21_accuracy_10: 0.8364 - dense_21_accuracy_11: 0.8589 - dense_21_accuracy_12: 0.8828 - dense_21_accuracy_13: 0.9052 - dense_21_accuracy_14: 0.9221 - dense_21_accuracy_15: 0.9328 - dense_21_accuracy_16: 0.9470 - dense_21_accuracy_17: 0.9517 - dense_21_accuracy_18: 0.9576 - dense_21_accuracy_19: 0.9642
Epoch 2/5
10000/10000 [==============================] - 100s 10ms/step - loss: 44.8126 - dense_21_loss: 0.4072 - dense_21_accuracy: 0.0132 - dense_21_accuracy_1: 0.1463 - dense_

In [ ]:
import math
def beam_search_decoder(predictions, top_k = 3):

    output_sequences = [([], 0)]
    
    for token_probs in predictions:
        new_sequences = []
        token_probs = token_probs.reshape(5000 ,)
        for old_seq, old_score in output_sequences:
            for char_index in range(len(token_probs)):
                new_seq = old_seq + [char_index]
                new_score = old_score + math.log(token_probs[char_index])
                new_sequences.append((new_seq, new_score))
                
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:top_k]
        
    return output_sequences

In [ ]:
def encode(sentence):
  seqn = tokenizer.texts_to_sequences(sentence)
  inputco = pad_sequences(seqn, padding='post', maxlen =20, truncating='post')
  return inputco

In [ ]:
def decode(context):
  outputprd = model.predict([context, np.zeros((1, 256))])
  output = beam_search_decoder(outputprd, 3)
  return output

In [ ]:
sentence = [("startseq where are you working? endseq")]
print(type(sentence))
context = encode(sentence)
print(context)
resp = decode(context)

<class 'list'>
[[  2 101  27   4 434   3   0   0   0   0   0   0   0   0   0   0   0   0
    0   0]]


In [ ]:
print(resp)

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -9.251278763711607), ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -9.680468917584045), ([6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -9.714830070591267)]


In [ ]:
index_to_word = {}
for word,index in word_index.items():
  index_to_word[index] = word

In [ ]:
print(index_to_word[6])

i


In [ ]:
sentence = [("startseq hey it's been a long time endseq")]
context = encode(sentence)
resp = decode(context)
print(resp)

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -9.826397868719901), ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -10.259679507668732), ([6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -10.292555449511473)]


In [ ]:
sentence = [("startseq are you free endseq")]
context = encode(sentence)
resp = decode(context)
print(resp)

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -8.977476341425914), ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -9.40553069101828), ([6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -9.441598680263144)]
